In [ ]:
import os, glob, cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

os.environ["CUDA_VISIBLE_DEVICES"] = "7"
import keras
from keras.layers import Input
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications import DenseNet121, InceptionV3, VGG16, VGG19
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import SGD
from keras.utils import multi_gpu_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras import backend as K

def recall(y_target, y_pred): # sensitivity, recall
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall



In [ ]:
pwd

'/Data/voucher/GP_Lee/ppm/02_code'

In [ ]:
PATH = '../01_data/PPM_data_aug/'
label_folder = os.listdir(PATH)
label_li = [float(l.split('_')[-1]) for l in label_folder if not 'csv' in l]

ValueError: could not convert string to float: '10.jpg'

In [23]:
row, col = 224,224

In [26]:
img_path_li = list()
data_img = [[],[],[]]
data_label = [[],[],[]]

for label_path in label_folder[:-1]:
    label = float(label_path.split('_')[-1])
    img_path_tmp = sorted(glob.glob(PATH+label_path+'/*'))
#     print('test')
    for path in img_path_tmp[-4:]:
#         print(path)
        img = cv2.resize(cv2.imread(path), (row, col))
        data_img[2].append(img)
        data_label[2].append(label//10-1)

In [27]:
test_x, test_y = np.array(data_img[2]), np.array(data_label[2])

print(test_x.shape, test_y.shape)

(80, 224, 224, 3) (80,)


In [28]:
test_x=preprocess_input(test_x)

# test_y = test_y/200

In [29]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import r2_score

model_name = 'class_exp2_vgg16'

# model = keras.models.load_model('../{}.h5'.format(model_name))
# ypred = model.predict(test_x)
# print(model.evaluate(test_x, test_y))
# print("MSE: %.4f" % mean_squared_error(test_y, ypred))
# print("MAE: %.4f" % mean_absolute_error(test_y, ypred))


print('*'*30)
print('Best model evaluate')
print('*'*30)
model_best = keras.models.load_model('../checkpoint/{}.h5'.format(model_name), custom_objects={'recall':recall})
ypred_best = model_best.predict(test_x)

pred_label=[]
for i,v in enumerate(ypred_best):
    pre_ans=v.argmax()
    pred_label.append(pre_ans)
    
cm=confusion_matrix(test_y,pred_label)
print(cm)
acc=accuracy_score(test_y,pred_label)
print(acc)

# tp,fn,fp,tn=cm.ravel()
# # print(tp,fn,fp,tn)

# acc=accuracy_score(test_y,pred_label)
# sensitivity=tp/(tp+fn)
# specificity=tn/(tn+fp)
# precision=tp/(tp+fp)
# F1_score=2*precision*sensitivity/(precision+sensitivity)

# y_pred1=np.load(pred_npy)
# # fp,tp,_=roc_curve(test_y,ypred_best[:,1])

# print('accuuacy : '+str(acc))
# print('sensitivity(recall) : '+str(sensitivity))
# print('specificity : '+str(specificity))
# print('precision : '+str(precision))
# print('F1_score : '+str(F1_score))
# # print('AUC score : {}'.format(round(auc(fp,tp),2)))

******************************
Best model evaluate
******************************
[[4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4]]
1.0


In [30]:
label_li_str = [str(i) for i in label_li]

In [31]:
confusion = confusion_matrix(test_y,pred_label)
print('Confusion Matrix\n')
print(confusion)

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(test_y, pred_label)))

print('Micro Precision: {:.2f}'.format(precision_score(test_y, pred_label, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(test_y, pred_label, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(test_y, pred_label, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(test_y, pred_label, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(test_y, pred_label, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(test_y, pred_label, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(test_y, pred_label, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(test_y, pred_label, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(test_y, pred_label, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(test_y, pred_label, target_names=label_li_str))


Confusion Matrix

[[4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4]]

Accuracy: 1.00

Micro Precision: 1.00
Micro Recall: 1.00
Micro F1-score: 1.00

Macro Precision: 1.00
Macro Recall: 1.00
